In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.optim as optim

import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

from sklearn import preprocessing

torch.set_num_threads(8)
os.environ['MKL_NUM_THREADS'] = "8"
os.environ['OMP_NUM_THREADS'] = "8"
OMP_NUM_THREADS=8
MKL_NUM_THREADS=8

import torch.backends.cudnn as cudnn
cudnn.benchmark = True

In [26]:
# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transform = transform=transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, 4),
            transforms.ToTensor(),
            normalize,
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transforms.Compose([transforms.ToTensor(), normalize]))

testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
trainset.train_data.shape

(50000, 32, 32, 3)

In [34]:
gpu=False

def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 2 every 30 epochs"""
    lr = 0.05 * (0.5 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def train_eval(net,n_epochs=50):
    C1,C2,C3=[],[],[]
    
    losses=[]
    if gpu==True:
        criterion = nn.CrossEntropyLoss().cuda()
    else:
        criterion = nn.CrossEntropyLoss()
    for epoch in range(n_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs
            inputs, labels = data

            # wrap them in Variable
            if gpu==True:
                inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)
            # forward + backward + optimize
            outputs = net(inputs)

            #optimizer = optim.SGD(filter(lambda p: p.requires_grad, net.parameters()), lr=0.001, momentum=0.9)
            
            optimizer = optim.SGD(filter(lambda p: p.requires_grad, net.parameters()), 
                                  lr=0.05, momentum=0.9, weight_decay=5e-4)
            adjust_learning_rate(optimizer, epoch)
            
            # zero the parameter gradients
            optimizer.zero_grad()

            #l1 = torch.abs(fcw).sum()
            #print('CRIT', criterion(outputs, labels))
            #print('L1', l1)

            loss = criterion(outputs, labels) #+ 0.001*l1
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.data[0]
            if i % 200 == 199:    # print every 2000 mini-batches
                #print('[%d, %5d] loss: %.3f' %
                #      (epoch + 1, i + 1, running_loss / 2000))
                losses.append(running_loss/ 2000)
                running_loss = 0.0
                C1.append(net3.conv1.weight.grad)
                C2.append(net3.conv2.weight.grad)
                C3.append(net3.conv3.weight.grad)
                #print('L1 = %s'%l1)        
    print('Finished Training')
    plt.plot(losses)

    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    for data in testloader:
        images, labels = data
        if gpu==True:                                 
            outputs = net(Variable(images).cuda())
            _, predicted = torch.max(outputs.data.cpu(), 1)
        else:
            outputs = net(Variable(images))
            _, predicted = torch.max(outputs.data, 1)                               
        c = (predicted == labels).squeeze()
        for i in range(8):
            label = labels[i]
            class_correct[label] += c[i]
            class_total[label] += 1


    for i in range(10):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))
    print np.sum(class_correct)/np.sum(class_total)
    
    return C1,C2,C3

## 3 layer randCNN 

In [32]:
class rand_Net3(nn.Module):
    def __init__(self):
        super(rand_Net3, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3,padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3,padding=1)
        self.conv3 = nn.Conv2d(128, 128, 3,padding=1)
        self.fc = nn.Linear(4096/2, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),kernel_size=2, stride=2)
        x = F.max_pool2d(F.relu(self.conv2(x)),kernel_size=2, stride=2)
        x = F.max_pool2d(F.relu(self.conv3(x)),kernel_size=2, stride=2)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = F.log_softmax(x)
        return x

In [35]:
net3 = rand_Net3()
#net3.cuda()
C1,C2,C3 = train_eval(net3,n_epochs=10)

/Users/yuki/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app
Process Process-16:
  File "/Users/yuki/anaconda2/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
Process Process-15:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/yuki/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/yuki/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
  File "/Users/yuki/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run


KeyboardInterrupt: 

  File "/Users/yuki/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
    self._target(*self._args, **self._kwargs)
  File "/Users/yuki/anaconda2/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
    r = index_queue.get()
  File "/Users/yuki/anaconda2/lib/python2.7/multiprocessing/queues.py", line 378, in get
  File "/Users/yuki/anaconda2/lib/python2.7/multiprocessing/queues.py", line 376, in get
    return recv()
    racquire()
  File "/Users/yuki/anaconda2/lib/python2.7/site-packages/torch/multiprocessing/queue.py", line 21, in recv
KeyboardInterrupt
    buf = self.recv_bytes()
KeyboardInterrupt


# try deeper network

In [47]:
#vgg11 [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']
   
class rand_vgg11(nn.Module):
    def __init__(self):
        super(rand_vgg11, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3,padding=1)
        
        self.conv2 = nn.Conv2d(64, 128*2, 3,padding=1)
        
        self.conv3 = nn.Conv2d(128*2, 256*2, 3,padding=1)
        self.conv4 = nn.Conv2d(256*2, 256*2, 3,padding=1)
                
        self.conv5 = nn.Conv2d(256*2, 512*4, 3,padding=1)
        self.conv6 = nn.Conv2d(512*4, 512*4, 3,padding=1)
        
        self.conv7 = nn.Conv2d(512*4, 512*4, 3,padding=1)
        self.conv8 = nn.Conv2d(512*4, 512*4, 3,padding=1)

        self.fc = nn.Linear(512*4, 10)
        for p in self.conv1.parameters():
            p.requires_grad = False
        for p in self.conv2.parameters():
            p.requires_grad = False
        for p in self.conv3.parameters():
            p.requires_grad = False
        for p in self.conv4.parameters():
            p.requires_grad = False
        for p in self.conv5.parameters():
            p.requires_grad = False
        for p in self.conv6.parameters():
            p.requires_grad = False
        for p in self.conv7.parameters():
            p.requires_grad = False
        for p in self.conv8.parameters():
            p.requires_grad = False
            
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),kernel_size=2, stride=2)
        x = F.max_pool2d(F.relu(self.conv2(x)),kernel_size=2, stride=2)
        
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(F.relu(self.conv4(x)),kernel_size=2, stride=2)
        
        x = F.relu(self.conv5(x))
        x = F.max_pool2d(F.relu(self.conv6(x)),kernel_size=2, stride=2)
        
        x = F.relu(self.conv7(x))
        x = F.max_pool2d(F.relu(self.conv8(x)),kernel_size=2, stride=2)
        
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x, self.fc.weight

In [ ]:
netvgg11 = rand_vgg11()
criterion = nn.CrossEntropyLoss() 
losses=[]
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # forward + backward + optimize
        outputs, fcw = netvgg11(inputs)
                
        optimizer = optim.SGD(filter(lambda p: p.requires_grad, netvgg11.parameters()), lr=0.001, momentum=0.9)
        # zero the parameter gradients
        optimizer.zero_grad()
        
        #l1 = torch.abs(fcw).sum()
        #print('CRIT', criterion(outputs, labels))
        #print('L1', l1)
        
        loss = criterion(outputs, labels) #+ 0.001*l1
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    losses.append(loss.data[0])
plt.plot(losses)
print('Finished Training')

In [32]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    outputs,_ = netvgg11(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(8):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print np.sum(class_correct)/np.sum(class_total)

Accuracy of plane :  0 %
Accuracy of   car :  0 %
Accuracy of  bird :  0 %
Accuracy of   cat :  0 %
Accuracy of  deer : 100 %
Accuracy of   dog :  0 %
Accuracy of  frog :  0 %
Accuracy of horse :  0 %
Accuracy of  ship :  0 %
Accuracy of truck :  0 %
0.1


## Compare to pixel by pixel Lasso classification

In [17]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=7000,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=500,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [19]:
from sklearn import linear_model,metrics,preprocessing
from sklearn.ensemble import RandomForestClassifier

print('compute purely on pixels')
for ty,data in enumerate(trainloader):
    images, labels = data
    for alpha in [1]:#np.logspace(0,3,10):
        print alpha
        reg = linear_model.LogisticRegression(penalty='l1',solver ='saga',C = alpha,n_jobs=6,max_iter=3)
        rf = RandomForestClassifier(n_estimators = 300,n_jobs=6)
        reg.fit(images[:,:,:,:].numpy().reshape(7000,-1) , labels) 
        rf.fit(images[:,:,:,:].numpy().reshape(7000,-1) , labels) 
    for mod in [reg,rf]:
        print ("-----")
        class_correct = list(0. for i in range(10))
        class_total = list(0. for i in range(10))
        for a,data in enumerate(testloader):
            imagesp, labelsp = data
            predicted = mod.predict(imagesp[:,:,:,:].numpy().reshape(500,-1))
            c = (predicted == labelsp).squeeze()
            for i in range(500):
                label = labelsp[i]
                class_correct[label] += c[i]
                class_total[label] += 1
            if a>=4:
                break 
        for i in range(10):
            print('Accuracy of %5s : %2d %%' % (
                classes[i], 100 * class_correct[i] / class_total[i]))
        print np.sum(class_correct)/np.sum(class_total)
    break



compute purely on pixels
1
-----
Accuracy of plane : 43 %
Accuracy of   car : 43 %
Accuracy of  bird : 26 %
Accuracy of   cat : 17 %
Accuracy of  deer : 28 %
Accuracy of   dog : 27 %
Accuracy of  frog : 48 %
Accuracy of horse : 48 %
Accuracy of  ship : 53 %
Accuracy of truck : 41 %
0.3804
-----
Accuracy of plane : 46 %
Accuracy of   car : 46 %
Accuracy of  bird : 33 %
Accuracy of   cat : 21 %
Accuracy of  deer : 32 %
Accuracy of   dog : 36 %
Accuracy of  frog : 51 %
Accuracy of horse : 45 %
Accuracy of  ship : 66 %
Accuracy of truck : 51 %
0.4368
